In [2]:
import pandas as pd
import numpy as np

In [3]:
ls data/sports_scores/

astros_1A10.csv*   Rice_10H80.csv*    Tex_15E40.csv*
dynamo_10H30.csv*  rockets_1A10.csv*  UH_10H50.csv*


In [4]:
ls data/crime_beats/

crime_10H30_10_17.csv*  crime_10H80_10_17.csv*  crime_1A10_10_17.csv*
crime_10H50_10_17.csv*  crime_15E40_10_17.csv*


In [5]:
games = 'data/sports_scores/Tex_15E40.csv'
crime = 'data/crime_beats/crime_15E40_10_17.csv'

## scores dataframes

In [7]:
df = pd.read_csv(games)


## split date and time from column scheduled
s_date = pd.DatetimeIndex(df['scheduled'])
df['Date'] = s_date.date
df['time'] = s_date.time
del df['scheduled']

# set date as datetime and index
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date').sort_index(ascending=True)






## win column
df['win'] =  np.where(df['scoring.home_points'] > df['scoring.away_points'],1,0)

# only oposing name and win boolean
df = df[['away.alias','win']]

# show

df.head()

,away.alias,win
Date,,
2010-01-03,NE,1
2010-09-12,IND,1
2010-09-26,DAL,0
2010-10-10,NYG,0
2010-10-17,KC,1


## crime dataframe

In [8]:
crime_df = pd.read_csv(crime,index_col='Date').sort_index(ascending=True)

## only crimes after 2009
crime_df = crime_df[crime_df.year >=2010]

# selectd columns
crime_df = crime_df[['OffenseType','Premise','NumOffenses','Hour','weekday']]

# count Offense type given date
date_crimes = crime_df.groupby(crime_df.index)['OffenseType',].count()


## show
date_crimes.head()

,OffenseType
Date,
2010-01-01,2
2010-01-02,2
2010-01-03,5
2010-01-04,2
2010-01-05,4


## Days dataframe

In [9]:
days = pd.date_range(start='01/01/2010', end='12/30/2017')
days = pd.DataFrame(days)

days.columns = ['days']
days = days.set_index('days').sort_index(ascending=True)

days.head()

""
days
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


## Merge days dataframe and  crimes

In [10]:
calendar_crimes = pd.merge(days,date_crimes,  left_index=True, right_index=True, how='left')
calendar_crimes.head()

,OffenseType
days,
2010-01-01,2.0
2010-01-02,2.0
2010-01-03,5.0
2010-01-04,2.0
2010-01-05,4.0


## Merge calendar_crimes with scores

In [12]:
merge_data = pd.merge(calendar_crimes,df,  left_index=True, right_index=True, how='left')

# change column names
merge_data.columns = ['offenses','away_team','win']

merge_data.head()

,offenses,away_team,win
days,,,
2010-01-01,2.0,NaN,NaN
2010-01-02,2.0,NaN,NaN
2010-01-03,5.0,NE,1.0
2010-01-04,2.0,NaN,NaN
2010-01-05,4.0,NaN,NaN


## Display non nans

In [13]:
merge_data.dropna(subset=['away_team', 'win'])

,offenses,away_team,win
days,,,
2010-01-03,5.0,NE,1.0
2010-09-12,7.0,IND,1.0
2010-09-26,13.0,DAL,0.0
2010-10-10,5.0,NYG,0.0
2010-10-17,6.0,KC,1.0
2010-11-07,4.0,SD,0.0
2010-11-28,9.0,TEN,1.0
2010-12-14,5.0,BAL,0.0
2011-01-02,6.0,JAC,1.0


In [14]:
merge_data.to_csv('data/final/Texans1017_final.csv')